<a href="https://colab.research.google.com/github/VishalPrem1994/AIGenPlayGround/blob/main/TFAutoModelForTokenClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
!pip install transformers -U
!pip install accelerate -U

In [ ]:
from transformers import AutoProcessor, TFAutoModelForTokenClassification, Trainer
import accelerate
import transformers

### Inference

In [ ]:
|from transformers import AutoProcessor, TFAutoModelForTokenClassification, Trainer
from datasets import load_dataset

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
model = TFAutoModelForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=7)

dataset = load_dataset("nielsr/funsd-layoutlmv3", split="test")
example = dataset[0]
image = example["image"]
words = example["tokens"]
boxes = example["bboxes"]
word_labels = example["ner_tags"]

encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="tf")

outputs = model(**encoding)
# loss = outputs.loss
# logits = outputs.logits

All model checkpoint layers were used when initializing TFLayoutLMv3ForTokenClassification.

Some layers of TFLayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['dropout', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.num_labels)

7


In [ ]:
outputs.logits.numpy()

array([[[ 0.43296117, -0.59000134, -0.06979164, ..., -0.48452416,
          0.27750528, -0.00721444],
        [ 0.19474994, -0.25761545,  0.09597155, ...,  0.00975339,
          0.12363185,  0.01789424],
        [ 0.07014784, -0.2321267 , -0.05487693, ..., -0.18979509,
          0.06941003,  0.12577416],
        ...,
        [-0.2288545 , -0.24830306,  0.42925757, ..., -0.26319492,
         -0.24275087, -0.01112129],
        [-0.00517309, -0.4841121 ,  0.12616858, ..., -0.37430978,
          0.21132559, -0.29877675],
        [ 0.43360245, -0.5920362 , -0.06913602, ..., -0.4860724 ,
          0.27816156, -0.01015994]]], dtype=float32)

In [ ]:
torch.softmax(torch.tensor(outputs.logits.numpy()),dim=-1)

tensor([[[0.2344, 0.0843, 0.1418,  ..., 0.0937, 0.2007, 0.1509],
         [0.1839, 0.1170, 0.1666,  ..., 0.1529, 0.1713, 0.1541],
         [0.1729, 0.1278, 0.1525,  ..., 0.1333, 0.1727, 0.1828],
         ...,
         [0.1288, 0.1263, 0.2487,  ..., 0.1244, 0.1270, 0.1601],
         [0.1669, 0.1034, 0.1904,  ..., 0.1154, 0.2073, 0.1245],
         [0.2347, 0.0842, 0.1420,  ..., 0.0936, 0.2009, 0.1506]]])

### training

In [ ]:
!pip install transformers datasets
!pip install transformers -U
!pip install accelerate -U

In [ ]:
from transformers import AutoProcessor, TFAutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, Image
import accelerate
import transformers

In [ ]:
datasets = load_dataset("nielsr/funsd")
datasets

In [ ]:
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)

In [ ]:
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id

In [ ]:
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

def preprocess_data(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']]
  words = examples['words']
  boxes = examples['bboxes']
  word_labels = examples['ner_tags']

  encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,padding="max_length", truncation=True, return_tensors="tf")

  return encoded_inputs

train_dataset = datasets['train'].map(preprocess_data, batched=True, remove_columns=datasets['train'].column_names,
                                      features=features)
test_dataset = datasets['test'].map(preprocess_data, batched=True, remove_columns=datasets['test'].column_names,
                                      features=features)

In [ ]:
model = TFAutoModelForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=7)

In [ ]:
args = TrainingArguments(
    output_dir="layoutlmv2-finetuned-funsd-v2", # name of directory to store the checkpoints
    max_steps=1000, # we train for a maximum of 1,000 batches
    warmup_ratio=0.1, # we warmup a bit
    fp16=True, # we use mixed precision (less memory consumption)
    push_to_hub=False, # after training, we'd like to push our model to the hub
)

In [ ]:
trainer = Trainer(model=model,args=None,train_dataset=None,eval_dataset=None)

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
model = TFAutoModelForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=7)

dataset = load_dataset("nielsr/funsd-layoutlmv", split="test")
example = dataset[0]
image = example["image"]
words = example["tokens"]
boxes = example["bboxes"]
word_labels = example["ner_tags"]

encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="tf")

outputs = model(**encoding)
# loss = outputs.loss
# logits = outputs.logits

In [ ]:
from PIL import Image
dataset = load_dataset("nielsr/funsd-layoutlmv3", split="test")
example = dataset[0]
image = example["image"]
image.copy()

In [ ]:

torch.softmax(torch.tensor(outputs.logits.numpy()),dim=-1)